In [ ]:
#GAN
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, RepeatVector, TimeDistributed, LeakyReLU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import tensorflow as tf

# Define a function to evaluate models and store results
def evaluate_model(model_name, y_true, y_pred, y_score, fit_time, predict_time):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_score) if y_score is not None else np.nan

    return {
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': roc_auc
    }

# Initialize a DataFrame to store results
results = pd.DataFrame(columns=[
    'Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC'

])

# Load and prepare data
data = pd.read_csv('/content/14bus_fdia_combined_dataset_overall.csv')

# Feature engineering
X = data.drop(['attack', 'bus', 'load_percentage', 'varied_load', 'voltage_increase', 'angle_increase'], axis=1)
y = data['attack']

# Split data (keeping temporal order for LSTM)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# MinMax scaler for GANs
minmax_scaler = MinMaxScaler(feature_range=(-1, 1))
X_train_minmax = minmax_scaler.fit_transform(X_train)
X_test_minmax = minmax_scaler.transform(X_test)


# ======================
# 3. GAN-based Detection
# ======================
print("\nTraining GAN models...")

# Common GAN components
def build_generator(latent_dim, output_dim):
    model = Sequential([
        Dense(128, input_dim=latent_dim),
        LeakyReLU(alpha=0.2),
        Dense(256),
        LeakyReLU(alpha=0.2),
        Dense(output_dim, activation='tanh')
    ])
    return model

def build_discriminator(input_dim):
    model = Sequential([
        Dense(256, input_dim=input_dim),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(128),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    return model

# ======================
# 3.1  GAN
# ======================
def train_gan(X_train, latent_dim=100, epochs=1400, batch_size=64):
    input_dim = X_train.shape[1]

    # Build and compile discriminator
    discriminator = build_discriminator(input_dim)
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

    # Build generator
    generator = build_generator(latent_dim, input_dim)

    # Combined model
    z = Input(shape=(latent_dim,))
    generated = generator(z)
    discriminator.trainable = False
    validity = discriminator(generated)
    combined = Model(z, validity)
    combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Train discriminator
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_samples = X_train[idx]

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_samples = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_samples, valid)
        d_loss_fake = discriminator.train_on_batch(gen_samples, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = combined.train_on_batch(noise, valid)

        if epoch % 100 == 0:
            print(f"Epoch {epoch} [D loss: {d_loss[0]:.4f}, acc: {100*d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")

    return generator, discriminator

print("\nTraining Standard GAN...")
gan_generator, gan_discriminator = train_gan(X_train_minmax)

# Anomaly detection with GAN
def gan_anomaly_detection(X_test, generator, discriminator, latent_dim=100, n_samples=1000):
    # Generate synthetic normal samples
    noise = np.random.normal(0, 1, (n_samples, latent_dim))
    generated_samples = generator.predict(noise)

    # Calculate discriminator scores for real and generated samples
    real_scores = discriminator.predict(X_test)
    gen_scores = discriminator.predict(generated_samples)

    # Calculate anomaly scores
    anomaly_scores = 1 - real_scores.flatten()
    threshold = np.percentile(1 - gen_scores.flatten(), 95)

    return anomaly_scores, threshold

gan_scores, gan_threshold = gan_anomaly_detection(X_test_minmax, gan_generator, gan_discriminator)
gan_pred = (gan_scores > gan_threshold).astype(int)

print("\nGAN Results:")
print(classification_report(y_test, gan_pred))
print(f"Accuracy: {accuracy_score(y_test, gan_pred):.4f}")
print(f"precision: {precision_score(y_test, gan_pred):.4f}")
print(f"recall: {recall_score(y_test, gan_pred):.4f}")
print(f"f1-score: {f1_score(y_test, gan_pred):.4f}")

# ======================
# ROC Curve for GAN
# ======================

# Calculate ROC-AUC
gan_roc_auc = roc_auc_score(y_test, gan_scores)

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y_test, gan_scores)

print(f"ROC-AUC Score: {gan_roc_auc:.4f}")

# Plot ROC Curve
plt.figure()
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - GAN")
plt.show()
